In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('data_preprocessed.csv')

In [3]:
data.head()

,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,0.0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590,1
1,1.0,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809,1
2,2.0,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,0
3,3.0,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723,1
4,4.0,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,1


In [4]:
targets = np.array(data['Quality'])
targets

array([1, 1, 0, ..., 0, 1, 1], dtype=int64)

In [6]:
targets.sum()/targets.shape[0]

0.501

In [8]:
data.iloc[:,:-1]

,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity
0,0.0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590
1,1.0,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809
2,2.0,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636
3,3.0,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723
4,4.0,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984
...,...,...,...,...,...,...,...,...
3995,3995.0,0.059386,-1.067408,-3.714549,0.473052,1.697986,2.244055,0.137784
3996,3996.0,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235
3997,3997.0,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611
3998,3998.0,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229720


In [12]:
inputs = data.iloc[:, 1:-1]

In [13]:
inputs

,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity
0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590
1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809
2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636
3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723
4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984
...,...,...,...,...,...,...,...
3995,0.059386,-1.067408,-3.714549,0.473052,1.697986,2.244055,0.137784
3996,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235
3997,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611
3998,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229720


In [11]:
import tensorflow as tf

In [17]:
samples_count = inputs.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count


train_inputs = inputs[:train_samples_count]
train_targets = targets[:train_samples_count]

validation_inputs = inputs[train_samples_count:
                                    train_samples_count+validation_samples_count]
validation_targets = targets[train_samples_count:
                                      train_samples_count+validation_samples_count]

test_inputs = inputs[train_samples_count+validation_samples_count:]
test_targets = targets[train_samples_count+validation_samples_count:]

In [19]:
np.savez('train_data', inputs=train_inputs, targets=train_targets)
np.savez('validation_data',
         inputs=validation_inputs, targets=validation_targets)
np.savez('test_data', inputs=test_inputs, targets=test_targets)

In [21]:
npz = np.load('train_data.npz')

train_inputs, train_targets = npz['inputs'].astype(
    np.float16), npz['targets'].astype(np.int32)

npz = np.load('validation_data.npz')

validation_inputs, validation_targets = npz['inputs'].astype(
    np.float16), npz['targets'].astype(np.int32)

npz = np.load('test_data.npz')

test_inputs, test_targets = npz['inputs'].astype(
    np.float16), npz['targets'].astype(np.int32)

In [25]:
test_targets.shape

(400,)

In [33]:
input_size = 7
output_size = 2
hidden_layer_size = 150

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(
                                hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(
                                hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(
                                output_size, activation='softmax')
                            ])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

BATCH_SIZE = 100

MAX_EPOCHS = 50

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs, train_targets, batch_size = BATCH_SIZE, epochs= MAX_EPOCHS, callbacks=[early_stopping], validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/50
32/32 - 1s - loss: 0.4701 - accuracy: 0.7684 - val_loss: 0.3733 - val_accuracy: 0.8350 - 845ms/epoch - 26ms/step
Epoch 2/50
32/32 - 0s - loss: 0.3408 - accuracy: 0.8494 - val_loss: 0.3225 - val_accuracy: 0.8575 - 101ms/epoch - 3ms/step
Epoch 3/50
32/32 - 0s - loss: 0.3054 - accuracy: 0.8612 - val_loss: 0.2898 - val_accuracy: 0.8775 - 84ms/epoch - 3ms/step
Epoch 4/50
32/32 - 0s - loss: 0.2806 - accuracy: 0.8763 - val_loss: 0.2766 - val_accuracy: 0.8925 - 108ms/epoch - 3ms/step
Epoch 5/50
32/32 - 0s - loss: 0.2658 - accuracy: 0.8906 - val_loss: 0.2606 - val_accuracy: 0.8800 - 93ms/epoch - 3ms/step
Epoch 6/50
32/32 - 0s - loss: 0.2536 - accuracy: 0.8978 - val_loss: 0.2554 - val_accuracy: 0.8925 - 91ms/epoch - 3ms/step
Epoch 7/50
32/32 - 0s - loss: 0.2446 - accuracy: 0.8997 - val_loss: 0.2456 - val_accuracy: 0.8925 - 88ms/epoch - 3ms/step
Epoch 8/50
32/32 - 0s - loss: 0.2393 - accuracy: 0.9019 - val_loss: 0.2403 - val_accuracy: 0.8875 - 105ms/epoch - 3ms/step
Epoch 9/50
32/32 - 

In [34]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

13/13 [==============================] - 0s 3ms/step - loss: 0.2580 - accuracy: 0.9175
